In [14]:
import pandas as pd
import glob

for file in glob.glob('./Dataset/*.csv'):


    # Load the CSV file into a DataFrame
    df = pd.read_csv(file)

    # Convert date formats in the `start_time` and `end_time` columns to be consistent (ISO format).
    df['start_time'] = pd.to_datetime(df['start_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['end_time'] = pd.to_datetime(df['end_time']).dt.strftime('%Y-%m-%d %H:%M:%S')

    # Exclude trips that exceeded 8 hours (480 minutes) in the `duration` column.
    df = df[df['duration'] <= 480]

    # Exclude trips with `plan_duration` values of 999, 150, or blank.
    df = df[~df['plan_duration'].isin([999, 150])]

    # Modify the `passholder_type` column:
    # Change "Flex Pass" to "Annual Pass".
    df['passholder_type'] = df['passholder_type'].replace('Flex Pass', 'Annual Pass')
    # If the `plan_duration` is 1 and the `passholder_type` is "Walk-up", change it to "One Day Pass".
    df.loc[(df['plan_duration'] == 1) & (df['passholder_type'] == 'Walk-up'), 'passholder_type'] = 'One Day Pass'

    # Exclude any trips with `start_station` or `end_station` values of 3000.
    df = df[(df['start_station'] != 3000) & (df['end_station'] != 3000)]

    # Identify bikes that have both "standard" and "smart" types
    bikes_with_both_types = df[df['bike_type'].isin(['standard', 'smart'])].groupby('bike_id')['bike_type'].nunique()
    bikes_with_both_types = bikes_with_both_types[bikes_with_both_types > 1].index.tolist()

    # Update their `bike_type` to "standard/smart"
    df.loc[df['bike_id'].isin(bikes_with_both_types), 'bike_type'] = 'standard/smart'

    df.head()

#put into new CSV file 
#df.to_csv("/path/to/save/cleaned_data.csv", index=False)

KeyError: 'bike_type'

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# First five rows of LA Metro bike_sharing_data 2016Q3-2021Q3
bike_sharing_data = pd.read_csv('./Dataset/metro-bike-share-trips-2017-q3.csv')
bike_sharing_data.head()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type
0,33404951,36,7/1/2017 0:09,7/1/2017 0:45,3018,34.043732,-118.260139,3018,34.043732,-118.260139,5996,0,Round Trip,Walk-up
1,33404950,13,7/1/2017 0:10,7/1/2017 0:23,3055,34.044159,-118.251579,3082,34.046520,-118.237411,5777,0,One Way,Walk-up
2,33404947,34,7/1/2017 0:11,7/1/2017 0:45,3018,34.043732,-118.260139,3018,34.043732,-118.260139,6342,0,Round Trip,Walk-up
3,33404948,34,7/1/2017 0:11,7/1/2017 0:45,3018,34.043732,-118.260139,3018,34.043732,-118.260139,6478,0,Round Trip,Walk-up
4,33404949,12,7/1/2017 0:11,7/1/2017 0:23,3055,34.044159,-118.251579,3082,34.046520,-118.237411,6411,0,One Way,Walk-up
